In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
import numpy as np
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

In [2]:
geo_path = "data/state_data/geo/ppp_geo.csv" #data as of 11/21/21
ppp_geo= pd.read_csv(geo_path)
ppp_geo.head()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.369867,NaN
1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.066165,NaN
2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.302559,NaN
3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.372550,NaN
4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.913680,NaN


In [13]:
ppp_geo_in = ppp_geo.loc[ppp_geo["BorrowerState"] =="RI"]
ppp_geo_in.iloc[:12330]


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
777541,777541,777541,777541,9179517702,5/1/20,165,PPP,"COASTAL MEDICAL, INC.",10 Davol Square Suite 300,Providnce,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,500 or more,"10 Davol Square Suite 300, Providnce, RI",41.8042,-71.420240,4.400700e+14
777542,777542,777542,777542,9713307002,4/9/20,165,PPP,BROWN MEDICINE,110 ELM ST,PROVIDENCE,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"110 ELM ST, PROVIDENCE, RI",41.817,-71.410620,4.400700e+14
777543,777543,777543,777543,5711067207,4/27/20,165,PPP,TACO INC,1160 CRANSTON ST,CRANSTON,...,333 Machinery,"3334 Ventilation, Heating, Air-Conditioning, a...",3334.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1160 CRANSTON ST, CRANSTON, RI",41.7951,-71.449750,4.400701e+14
777544,777544,777544,777544,4819187009,4/4/20,165,PPP,"UNIVERSITY ORTHOPEDICS, INC.",1 Kettle Point Avenue,EAST PROVIDENCE,...,999 Other,6211 Offices of Physicians,6211.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"1 Kettle Point Avenue, EAST PROVIDENCE, RI",41.7967,-71.375510,4.400701e+14
777545,777545,777545,777545,3895377102,4/12/20,165,PPP,"H. CARR & SONS, INC.",100 Royal Little Drive,PROVIDENCE,...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,e $5-10 million,250 to 499,"100 Royal Little Drive, PROVIDENCE, RI",41.8471,-71.413536,4.400700e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781328,781328,781328,781328,8379078305,1/29/21,165,PPS,GRAPHIC INNOVATIONS INC,380 Jefferson Blvd Ste C,Warwick,...,323 Printing & Publishing,3231 Printing and Related Support Activities,3231.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"380 Jefferson Blvd Ste C, Warwick, RI",41.7382,-71.439550,4.400302e+14
781329,781329,781329,781329,8836008607,3/25/21,165,PPS,NEW ENGLAND DRYWALL CO.,816 Middle Rd Unit 5,East Greenwich,...,999 Other,2383 Building Finishing Contractors,2383.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"816 Middle Rd Unit 5, East Greenwich, RI",41.6459,-71.489230,4.400302e+14
781330,781330,781330,781330,8960777006,4/9/20,165,PPP,WATER DAMAGE DRYING SERVICES INC.,12 GAVITT AVE,WESTERLY,...,999 Other,2389 Other Specialty Trade Contractors,2389.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12 GAVITT AVE, WESTERLY, RI",41.3833,-71.820860,4.400905e+14
781331,781331,781331,781331,9102547103,4/15/20,165,PPP,STORMTITE COMPANY INC.,1065 WARWICK AVE,WARWICK,...,999 Other,2361 Residential Building Construction,2361.0,RHODE ISLAND DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1065 WARWICK AVE, WARWICK, RI",41.747,-71.405410,4.400302e+14


In [3]:
ppp_geonull=ppp_geo.loc[ppp_geo["Lat"].isnull()] #457467
ppp_geonull.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
369,369,369,369,5048318601,3/20/21,1084,PPS,DELTA INDUSTRIAL SERVICES INC.,1229 RICHARDSON HWY,DELTA JUNCTION,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",NaN,NaN,NaN
428,428,428,428,5085327008,4/5/20,1084,PPP,"DELTA INDUSTRIAL SERVICES, INC.",1229 Richardson Hwy,DELTA JUNCTION,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1229 Richardson Hwy, DELTA JUNCTION, AK",NaN,NaN,NaN
1865,1865,1865,1865,4637628408,2/6/21,1084,PPS,MCKINLEY FIRE PROTECTION LLC,1706 Ship Ave,Anchorage,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 Ship Ave, Anchorage, AK",NaN,NaN,NaN
2274,2274,2274,2274,5337318401,2/8/21,1084,PPS,THE ALASKA CENTRE FOR CREATIVE RENEWAL LLC,46877 STILLPOINT TRL,HALIBUT COVE,...,999 Other,7211 Traveler Accommodation,7211.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"46877 STILLPOINT TRL, HALIBUT COVE, AK",NaN,NaN,NaN
2379,2379,2379,2379,9891427104,4/15/20,1084,PPP,MCKINLEY FIRE PROTECTION,1706 SHIP AVENUE,ANCHORAGE,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 SHIP AVENUE, ANCHORAGE, AK",NaN,NaN,NaN


In [18]:
ppp_geonull["BorrowerState"].value_counts()

TX    76183
NY    74112
FL    60040
MI    29806
WA    23754
CO    19741
MN    19730
WI    17354
IN    16601
MO    16350
LA    13668
OR    13456
UT     9295
IA     8295
NE     5804
HI     4980
NM     4485
ID     4416
ND     3222
MT     3144
PR     2952
SD     2695
OK     2535
WY     2139
SC     2045
MS     1589
NV     1569
KS     1485
GU      443
RI      414
VI      290
MP       82
NC       11
AL        8
PA        8
AK        5
AZ        4
TN        3
MA        3
NJ        3
ME        2
AR        2
IL        2
CT        1
OH        1
GA        1
WV        1
Name: BorrowerState, dtype: int64

In [19]:
ppp_togeoIN = ppp_geonull.loc[ppp_geonull["BorrowerState"] == "IN"]
ppp_togeoIN #13668

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
350732,350732,350732,350732,1266777200,4/15/20,562,PPP,SPECIALTY STEEL WORKS INCORPORATED,1412 150TH ST,HAMMOND,...,331 Primary Metals,3312 Steel Product Manufacturing from Purchase...,3312.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1412 150TH ST, HAMMOND, IN",NaN,NaN,NaN
350733,350733,350733,350733,2313017204,4/16/20,562,PPP,V GLOBAL HOLDINGS LLC,201 N. Illinois St. 1800,Indianapolis,...,325 Chemicals,3251 Basic Chemical Manufacturing,3251.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"201 N. Illinois St. 1800, Indianapolis, IN",NaN,NaN,NaN
350734,350734,350734,350734,2341197209,4/16/20,562,PPP,"DIVERSE STAFFING SERVICES, INC.",7135 Waldemar Drive,INDIANAPOLIS,...,999 Other,5613 Employment Services,5613.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"7135 Waldemar Drive, INDIANAPOLIS, IN",NaN,NaN,NaN
350735,350735,350735,350735,3129227104,4/11/20,507,PPP,"CROWN CORR, INC.",7100 W 21st Ave,GARY,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"7100 W 21st Ave, GARY, IN",NaN,NaN,NaN
350736,350736,350736,350736,5372347006,4/5/20,562,PPP,C.A. ADVANCED INC,311 S. indiana Ave,WAKARUSA,...,999 Other,2371 Utility System Construction,2371.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"311 S. indiana Ave, WAKARUSA, IN",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367328,367328,367328,367328,9384688310,1/30/21,562,PPS,RECOGNITION SERVICES INC,7998 Georgetown Rd Ste 200,Indianapolis,...,999 Other,"5418 Advertising, Public Relations, and Relate...",5418.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"7998 Georgetown Rd Ste 200, Indianapolis, IN",NaN,NaN,NaN
367329,367329,367329,367329,9490728302,1/30/21,562,PPS,HOLWAGER & HOLWAGER ATTORNEYS AT,1818 Main St,Beech Grove,...,999 Other,5411 Legal Services,5411.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1818 Main St, Beech Grove, IN",NaN,NaN,NaN
367330,367330,367330,367330,9571197108,4/15/20,562,PPP,D & D FARMS,8068 E County Road 100 N,Logansport,...,999 Other,1121 Cattle Ranching and Farming,1121.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"8068 E County Road 100 N, Logansport, IN",NaN,NaN,NaN
367331,367331,367331,367331,9715977705,5/1/20,562,PPP,DAVID E SCHWARTZ,12415 WINDSOR DR,CARMEL,...,999 Other,6241 Individual and Family Services,6241.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12415 WINDSOR DR, CARMEL, IN",NaN,NaN,NaN


In [7]:
#process to identify and code for unique addresses 
cols = ["LoanNumber","full_add"]
ppp_togeoIN_add = ppp_togeoIN[cols]
ppp_togeoIN_add

,LoanNumber,full_add
350732,1266777200,"1412 150TH ST, HAMMOND, IN"
350733,2313017204,"201 N. Illinois St. 1800, Indianapolis, IN"
350734,2341197209,"7135 Waldemar Drive, INDIANAPOLIS, IN"
350735,3129227104,"7100 W 21st Ave, GARY, IN"
350736,5372347006,"311 S. indiana Ave, WAKARUSA, IN"
...,...,...
367328,9384688310,"7998 Georgetown Rd Ste 200, Indianapolis, IN"
367329,9490728302,"1818 Main St, Beech Grove, IN"
367330,9571197108,"8068 E County Road 100 N, Logansport, IN"
367331,9715977705,"12415 WINDSOR DR, CARMEL, IN"


In [8]:
add = ppp_togeoIN["full_add"].astype(str).to_list()

In [9]:
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?form'
driver.visit(url)

not_found = []
lat = []
long = []
geo_id = []

i=0
for i in tqdm(range(len(add))):
        driver.find_by_id('_address_id').clear()
        driver.find_by_id('_address_id').fill(add[i])
        driver.find_by_id('proceed').click()
        
        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id="pl_gov_census_geo_geocoder_domain_AddressResult")
        
        for result in results:
            result_text = result.text
            result_lat = re.findall('[^-](\d{2}[.]+\d+)', result_text)
            result_long = re.findall('[-]\d{2}[.]+\d+', result_text) #WEST 3-digit Longitude
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('18')] #IN
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{add[i]} is {result_lat}, {result_long}, {fips_elements}")
                lat.append(result_lat[0])
                long.append(result_long[0])
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                lat.append(None)  
                long.append(None)
                geo_id.append(None)  
                print(f"{add[i]} is Not Found")
            except (TimeoutException):
                driver.navigate.refresh()
    
        sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/99.0.4844.51]


250 CROSS POINTE BLVD, EVANSVILLE, IN is Not Found
1200 Cutting Edge Dr., CHESTERTON, IN is Not Found
1 NOVAE PKWY, MARKLE, IN is Not Found
1301 E DOUGLAS RD, SUITE 200, MISHAWAKA, IN is Not Found
977 State Rt 46 East Suite C, Batesville, IN is Not Found
One America Square, Ste 2900, INDIANAPOLIS, IN is Not Found
ONE BEST DRIVE, FERDINAND, IN is Not Found
888 S. Harrison Ste. 900 0.0, Fort Wayne, IN is Not Found
P.O. Box 904, KOKOMO, IN is Not Found
173 COUNTY ROAD 525, AVON, IN is Not Found
1914 Happiness Way, LA PORTE, IN is Not Found
158 Le Mans Hall, NOTRE DAME, IN is Not Found
600 Corporation Drive, Suite 200, PENDLETON, IN is Not Found
310 WABASH ST, ASHLEY, IN is Not Found
977 State Rt 46 East Suite C, Batesville, IN is Not Found
3500 DePauw Boulevard, Suite 1090 0.0, Indianapolis, IN is Not Found
633 Avenue of Autos, FORT WAYNE, IN is Not Found
1001 West Brooklyn Street 0.0, Syracuse, IN is Not Found
415 NORTH 26TH ST SUITE 102, LAFAYETTE, IN is Not Found
5441 East Lincoln High

6401 South US Highway 41, TERRE HAUTE, IN is Not Found
4421 E.N. COUNTY LINE ROAD, CAMBY, IN is Not Found
16TH ST, INDIANAPOLIS, IN is Not Found
9701 OLD STATE ROAD 25, LAFAYETTE, IN is Not Found
2400 Enterprise Park Pl, Indianapolis, IN is Not Found
110 MENKE RD, TRAIL CREEK, IN is Not Found
1 INDIANA SQ STE 1800, INDIANAPOLIS, IN is Not Found
P O Box 238 421 W Canal St, WABASH, IN is Not Found
400 West 7th Street Suite 233, BLOOMINGTON, IN is Not Found
PO Box 965, EVANSVILLE, IN is Not Found
19319 COUNTY ROAD 8, MIDDLEBURY, IN is Not Found
2400 Enterprise Park Place, INDIANAPOLIS, IN is Not Found
8450 North Payne Road Suite 300, Indianapolis, IN is Not Found
2351 Production Ct, Richmond, IN is Not Found
6060 GUION ROAD 0.0, INDIANAPOLIS, IN is Not Found
7230 Engle Road, FORT WAYNE, IN is Not Found
1 Indiana SQ. STE 300, Indianapolis, IN is Not Found
5901 TECHNOLOGY CENTER DR, INDIANAPOLIS, IN is Not Found
300 SE RIVERSIDE DR Suite 100, EVANSVILLE, IN is Not Found
8090 W. 101st PL Sai

3500 DePauw Blvd Ste 2020, Indianapolis, IN is Not Found
8055 U.S. 31 South, INDIANAPOLIS, IN is Not Found
9401 S. INNOVATION DR, DALEVILLE, IN is Not Found
750 U.S. 31 North 0.0, Greenwood, IN is Not Found
711 MAIN ST, SCHERERVILLE, IN is Not Found
1251 Quaker Boulevard 0.0, Plainfield, IN is Not Found
5320 S Main St, South Bend, IN is Not Found
5320 S Main Street, SOUTH BEND, IN is Not Found
225 S East Street Suite 144, Indianapolis, IN is Not Found
10101 NORTH GRAY RD, INDIANAPOLIS, IN is Not Found
711 Main St, Schererville, IN is Not Found
1690 Commerce Dr, Bristol, IN is Not Found
5975 Castle Pkwy, Suite 250, INDIANAPOLIS, IN is Not Found
9333 North Meridian St, Suite 350, Indianapolis, IN is Not Found
101 W STATE ROAD 4, HUDSON, IN is Not Found
4211 CR 61 0.0, Butler, IN is Not Found
1800 Purdue Parkway, ANDERSON, IN is Not Found
6612 E 75th St Ste 450, Indianapolis, IN is Not Found
1156 E ROAD 800, PITTSBORO, IN is Not Found
2310 Toledo Rd, Elkhart, IN is Not Found
2310 TOLEDO R

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6511 STATE ROAD 205, LAOTTO, IN is Not Found
2255 Walter Glaub Dr, GRANGER, IN is Not Found
7225 Heritage Square Dr Ste 208, Granger, IN is Not Found
4631 LISBORN DR, CARMEL, IN is Not Found
3945 EAGLE CREEK PKWY STE A, INDIANAPOLIS, IN is Not Found
6640 Intech Blvd Suite 100, Indianapolis, IN is Not Found
3801 South St, Lafayette, IN is Not Found
6484 Central Blvd, Whitestown, IN is Not Found
1425 McShay Dr West Lafayette In, West Lafayette, IN is Not Found
1212 W Rappel Ave, Bloomington, IN is Not Found
1212 RAPPEL AVE, BLOOMINGTON, IN is Not Found
606 MAIN ST, PITTSBORO, IN is Not Found
1200 US HIGHWAY 41, SCHERERVILLE, IN is Not Found
4600 Albert S. White Dr Ste 200, WHITESTOWN, IN is Not Found
4474 E. Park 30 Dr., COLUMBIA CITY, IN is Not Found
6035 Sterling Creek Rd, PORTAGE, IN is Not Found
5719 LAWTON LOOP EAST DR STE 102, INDIANAPOLIS, IN is Not Found
1638 N 350W, JASPER, IN is Not Found
30809 Corwin Road, ELKHART, IN is Not Found
9400 Corporate Drive, INDIANAPOLIS, IN is Not 

2929 S 3RD PL, TERRE HAUTE, IN is Not Found
8739 Castle Park Dr Ste B, Indianapolis, IN is Not Found
8739 Castle Park Dr. Ste. B, INDIANAPOLIS, IN is Not Found
665 N MADISON ST, CROWN POINT, IN is Not Found
4034 Sourth US 35 PO Box 1766, LA PORTE, IN is Not Found
3506 HENKE ST, ELKHART, IN is Not Found
4630-8 JEFFERSON BLVD, FORT WAYNE, IN is Not Found
976 POUND DR, WARSAW, IN is Not Found
50 Countryside Dr, Markle, IN is Not Found
12953 PUBLISHERS DR, FISHERS, IN is Not Found
3061 N MORTON ST, FRANKLIN, IN is Not Found
1843 US  E, MICHIGAN CITY, IN is Not Found
1411 LINCOLN HWY E, NEW HAVEN, IN is Not Found
4421 E COUNTY LINE RD 4421 E COUNTY LINE RD, CAMBY, IN is Not Found
1 INDIANA SQ STE 2200, INDIANAPOLIS, IN is Not Found
P.O. Box 88, DALE, IN is Not Found
530 E LEXINGTON STE 175, ELKHART, IN is Not Found
624 State Rte.930 E. p.o. 245, NEW HAVEN, IN is Not Found
2855 Miller Dr. Suite 117, PLYMOUTH, IN is Not Found
1804 N MEMORIAL WAY, GREENSBURG, IN is Not Found
1675 South St Rd 9

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6035 STELLHORN RD, FORT WAYNE, IN is Not Found
2005 SAINT CHARLES ST STE 5, JASPER, IN is Not Found
300 HILTON DR, JEFFERSONVILLE, IN is Not Found
5031 E Lincoln Hwy, Merrillville, IN is Not Found
WORKERS UNION LOCAL 725, TERRE HAUTE, IN is Not Found
5700 W Minnesoate St Ste 1, INDIANAPOLIS, IN is Not Found
4918 E 550 S 4918 E 550 S, Whitestown, IN is Not Found
1101 Cumberland Crossing, VALPARAISO, IN is Not Found
PO Box 3966, SOUTH BEND, IN is Not Found
965 EMERSON PKWY, GREENWOOD, IN is Not Found
3435 N  4TH ST, NORTH VERNON, IN is Not Found
7260 GeorgetownRoad, Indianapolis, IN is Not Found
1721 Lincoln Highway East, NEW HAVEN, IN is Not Found
PO BOX 5041 115 N Farabee Dr, LAFAYETTE, IN is Not Found
3500 FRANCISCAN WAY MOB 4TH FLOOR STE 4E, MICHIGAN CITY, IN is Not Found
205 darrough dr, GREENWOOD, IN is Not Found
819 Mount Tabor Rd Ste 10, New Albany, IN is Not Found
3061 N Morton St, Franklin, IN is Not Found
642 N Opportunity Dr, Ste 101, COLUMBIA CITY, IN is Not Found
9660 East 

In [10]:
print(len(add))
print(len(lat))
print(len(long))
print(len(geo_id))

16601
16601
16601
16601


<b>Add remaining values (2801) to result_df(5461)

In [12]:
result_df = pd.DataFrame({"full_add":add, "Lat":lat, "Long":long, "FIPS_z": geo_id})
result_df 

,full_add,Lat,Long,FIPS_z
0,"1412 150TH ST, HAMMOND, IN",41.62311,-87.49383,[180890204001028]
1,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.16,[180973910011101]
2,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.25291,[180973102011027]
3,"7100 W 21st Ave, GARY, IN",41.58084,-87.424965,[180890103023014]
4,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.01523,[180390013011018]
...,...,...,...,...
16596,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.24093,[180973101051006]
16597,"1818 Main St, Beech Grove, IN",39.721546,-86.10313,[180973804042001]
16598,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.22518,[180179517001009]
16599,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,[180571110033009]


In [13]:
result_df['FIPS_z'] = result_df['FIPS_z'].str[0]
result_df 

,full_add,Lat,Long,FIPS_z
0,"1412 150TH ST, HAMMOND, IN",41.62311,-87.49383,180890204001028
1,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.16,180973910011101
2,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.25291,180973102011027
3,"7100 W 21st Ave, GARY, IN",41.58084,-87.424965,180890103023014
4,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.01523,180390013011018
...,...,...,...,...
16596,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.24093,180973101051006
16597,"1818 Main St, Beech Grove, IN",39.721546,-86.10313,180973804042001
16598,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.22518,180179517001009
16599,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,180571110033009


In [20]:
#mapping coordinates to NaN values in main 
m = dict(zip(result_df.full_add,result_df.Lat))
ppp_togeoIN.Lat = ppp_togeoIN.Lat.fillna(ppp_togeoIN.full_add.map(m))
n = dict(zip(result_df.full_add,result_df.Long))
ppp_togeoIN.Long = ppp_togeoIN.Long.fillna(ppp_togeoIN.full_add.map(n))
o = dict(zip(result_df.full_add,result_df.FIPS_z))
ppp_togeoIN.FIPS_z = ppp_togeoIN.FIPS_z.fillna(ppp_togeoIN.full_add.map(o))

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [21]:
ppp_togeoIN

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
350732,350732,350732,350732,1266777200,4/15/20,562,PPP,SPECIALTY STEEL WORKS INCORPORATED,1412 150TH ST,HAMMOND,...,331 Primary Metals,3312 Steel Product Manufacturing from Purchase...,3312.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1412 150TH ST, HAMMOND, IN",41.62311,-87.49383,180890204001028
350733,350733,350733,350733,2313017204,4/16/20,562,PPP,V GLOBAL HOLDINGS LLC,201 N. Illinois St. 1800,Indianapolis,...,325 Chemicals,3251 Basic Chemical Manufacturing,3251.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.16,180973910011101
350734,350734,350734,350734,2341197209,4/16/20,562,PPP,"DIVERSE STAFFING SERVICES, INC.",7135 Waldemar Drive,INDIANAPOLIS,...,999 Other,5613 Employment Services,5613.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.25291,180973102011027
350735,350735,350735,350735,3129227104,4/11/20,507,PPP,"CROWN CORR, INC.",7100 W 21st Ave,GARY,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"7100 W 21st Ave, GARY, IN",41.58084,-87.424965,180890103023014
350736,350736,350736,350736,5372347006,4/5/20,562,PPP,C.A. ADVANCED INC,311 S. indiana Ave,WAKARUSA,...,999 Other,2371 Utility System Construction,2371.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.01523,180390013011018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367328,367328,367328,367328,9384688310,1/30/21,562,PPS,RECOGNITION SERVICES INC,7998 Georgetown Rd Ste 200,Indianapolis,...,999 Other,"5418 Advertising, Public Relations, and Relate...",5418.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.24093,180973101051006
367329,367329,367329,367329,9490728302,1/30/21,562,PPS,HOLWAGER & HOLWAGER ATTORNEYS AT,1818 Main St,Beech Grove,...,999 Other,5411 Legal Services,5411.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1818 Main St, Beech Grove, IN",39.721546,-86.10313,180973804042001
367330,367330,367330,367330,9571197108,4/15/20,562,PPP,D & D FARMS,8068 E County Road 100 N,Logansport,...,999 Other,1121 Cattle Ranching and Farming,1121.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.22518,180179517001009
367331,367331,367331,367331,9715977705,5/1/20,562,PPP,DAVID E SCHWARTZ,12415 WINDSOR DR,CARMEL,...,999 Other,6241 Individual and Family Services,6241.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,180571110033009


In [23]:
cols = ["LoanNumber", "FIPS_z", "Lat", "Long"]
ppp_togeoIN_parsed = ppp_togeoIN[cols]
ppp_togeoIN_parsed.head()

,LoanNumber,FIPS_z,Lat,Long
350732,1266777200,180890204001028,41.62311,-87.49383
350733,2313017204,180973910011101,39.764225,-86.16
350734,2341197209,180973102011027,39.882576,-86.25291
350735,3129227104,180890103023014,41.58084,-87.424965
350736,5372347006,180390013011018,41.533318,-86.01523


In [24]:
ppp_geo = ppp_geo.merge(ppp_togeoIN_parsed, on="LoanNumber", how="left")
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat_x,Long_x,FIPS_z_x,FIPS_z_y,Lat_y,Long_y
0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.853682,-80.369867,NaN,NaN,NaN,NaN
1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,...,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.942332,-80.066165,NaN,NaN,NaN,NaN
2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,...,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.636067,-122.302559,NaN,NaN,NaN,NaN
3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,...,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.224719,-117.372550,NaN,NaN,NaN,NaN
4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,...,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN,NaN,NaN,NaN


In [25]:
ppp_geo["Lat"] = ppp_geo["Lat_y"].fillna(ppp_geo["Lat_x"])
ppp_geo["Long"] = ppp_geo["Long_y"].fillna(ppp_geo["Long_x"])
ppp_geo["FIPS_z"] = ppp_geo["FIPS_z_y"].fillna(ppp_geo["FIPS_z_x"])
ppp_geo = ppp_geo.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_y", "FIPS_z_x"], axis=1)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN


In [26]:
ppp_geo

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968552,968552,978144,977196,4395967002,4/3/20,897,PPP,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,CASPER,...,999 Other,6212 Offices of Dentists,6212.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"102 N. Kenwood, CASPER, WY",NaN,NaN,NaN
968553,968553,978145,977197,6985647108,4/14/20,897,PPP,"SWEETWATER COUNTY CHILD DEVELOPMENTAL CENTER, ...",1715 HITCHING POST DR,GREEN RIVER,...,999 Other,6244 Child Day Care Services,6244.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1715 HITCHING POST DR, GREEN RIVER, WY",NaN,NaN,NaN
968554,968554,978146,977198,7996438405,2/12/21,897,PPS,ELECTRICAL SYSTEMS OF WYOMING INC,1105 Adon Rd,Rozet,...,999 Other,2382 Building Equipment Contractors,2382.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1105 Adon Rd, Rozet, WY",NaN,NaN,NaN
968555,968555,978147,977199,9054647103,4/15/20,897,PPP,EDEN LIFE CARE,30 N. Gould Street Suite 4000,SHERIDAN,...,999 Other,6216 Home Health Care Services,6216.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"30 N. Gould Street Suite 4000, SHERIDAN, WY",NaN,NaN,NaN


In [27]:
ppp_geo.to_csv("data/state_data/geo/ppp_geo.csv")

In [28]:
ppp_geo_in = ppp_geo.loc[ppp_geo["BorrowerState"] =="IN"]
ppp_geo_in

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
350732,350732,350732,350732,1266777200,4/15/20,562,PPP,SPECIALTY STEEL WORKS INCORPORATED,1412 150TH ST,HAMMOND,...,331 Primary Metals,3312 Steel Product Manufacturing from Purchase...,3312.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1412 150TH ST, HAMMOND, IN",41.62311,-87.49383,180890204001028
350733,350733,350733,350733,2313017204,4/16/20,562,PPP,V GLOBAL HOLDINGS LLC,201 N. Illinois St. 1800,Indianapolis,...,325 Chemicals,3251 Basic Chemical Manufacturing,3251.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"201 N. Illinois St. 1800, Indianapolis, IN",39.764225,-86.16,180973910011101
350734,350734,350734,350734,2341197209,4/16/20,562,PPP,"DIVERSE STAFFING SERVICES, INC.",7135 Waldemar Drive,INDIANAPOLIS,...,999 Other,5613 Employment Services,5613.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"7135 Waldemar Drive, INDIANAPOLIS, IN",39.882576,-86.25291,180973102011027
350735,350735,350735,350735,3129227104,4/11/20,507,PPP,"CROWN CORR, INC.",7100 W 21st Ave,GARY,...,999 Other,"2381 Foundation, Structure, and Building Exter...",2381.0,ILLINOIS DISTRICT OFFICE,e $5-10 million,250 to 499,"7100 W 21st Ave, GARY, IN",41.58084,-87.424965,180890103023014
350736,350736,350736,350736,5372347006,4/5/20,562,PPP,C.A. ADVANCED INC,311 S. indiana Ave,WAKARUSA,...,999 Other,2371 Utility System Construction,2371.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"311 S. indiana Ave, WAKARUSA, IN",41.533318,-86.01523,180390013011018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367328,367328,367328,367328,9384688310,1/30/21,562,PPS,RECOGNITION SERVICES INC,7998 Georgetown Rd Ste 200,Indianapolis,...,999 Other,"5418 Advertising, Public Relations, and Relate...",5418.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"7998 Georgetown Rd Ste 200, Indianapolis, IN",39.898895,-86.24093,180973101051006
367329,367329,367329,367329,9490728302,1/30/21,562,PPS,HOLWAGER & HOLWAGER ATTORNEYS AT,1818 Main St,Beech Grove,...,999 Other,5411 Legal Services,5411.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1818 Main St, Beech Grove, IN",39.721546,-86.10313,180973804042001
367330,367330,367330,367330,9571197108,4/15/20,562,PPP,D & D FARMS,8068 E County Road 100 N,Logansport,...,999 Other,1121 Cattle Ranching and Farming,1121.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"8068 E County Road 100 N, Logansport, IN",40.779312,-86.22518,180179517001009
367331,367331,367331,367331,9715977705,5/1/20,562,PPP,DAVID E SCHWARTZ,12415 WINDSOR DR,CARMEL,...,999 Other,6241 Individual and Family Services,6241.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"12415 WINDSOR DR, CARMEL, IN",39.968594,-86.096794,180571110033009


In [32]:
null_check = ppp_geo_in.loc[ppp_geo_in["FIPS_z"].isnull()]
null_check

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
350737,350737,350737,350737,5434877008,4/5/20,562,PPP,"WHITE STALLION ENERGY, LLC",250 CROSS POINTE BLVD,EVANSVILLE,...,999 Other,2131 Support Activities for Mining,2131.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"250 CROSS POINTE BLVD, EVANSVILLE, IN",NaN,NaN,NaN
350749,350749,350749,350749,1683977300,4/28/20,562,PPP,"URSCHEL LABORATORIES, INC.",1200 Cutting Edge Dr.,CHESTERTON,...,333 Machinery,3332 Industrial Machinery Manufacturing,3332.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"1200 Cutting Edge Dr., CHESTERTON, IN",NaN,NaN,NaN
350750,350750,350750,350750,6084567002,4/6/20,562,PPP,NOVAE CORP,1 NOVAE PKWY,MARKLE,...,336 Transportation Equipment,3362 Motor Vehicle Body and Trailer Manufacturing,3362.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"1 NOVAE PKWY, MARKLE, IN",NaN,NaN,NaN
350755,350755,350755,350755,9771547100,4/15/20,562,PPP,SCHURZ COMMUNICATIONS INC,"1301 E DOUGLAS RD, SUITE 200",MISHAWAKA,...,999 Other,5152 Cable and Other Subscription Programming,5152.0,INDIANA DISTRICT OFFICE,e $5-10 million,250 to 499,"1301 E DOUGLAS RD, SUITE 200, MISHAWAKA, IN",NaN,NaN,NaN
350760,350760,350760,350760,9116507201,4/28/20,562,PPP,CINTEMP INC.,977 State Rt 46 East Suite C,Batesville,...,999 Other,5613 Employment Services,5613.0,INDIANA DISTRICT OFFICE,e $5-10 million,500 or more,"977 State Rt 46 East Suite C, Batesville, IN",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367294,367294,367294,367294,6309287101,4/14/20,562,PPP,MASTERTECH LLC,4800 BECK DR Unit A,ELKHART,...,999 Other,8111 Automotive Repair and Maintenance,8111.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4800 BECK DR Unit A, ELKHART, IN",NaN,NaN,NaN
367299,367299,367299,367299,6929428604,3/23/21,562,PPP,FIRST ACHIEVE THE CHALLENGES FOR EXCELLENCE L....,1311 North Arlington Ave Suite 209 209 209,Indianapolis,...,999 Other,6216 Home Health Care Services,6216.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1311 North Arlington Ave Suite 209 209 209, In...",NaN,NaN,NaN
367310,367310,367310,367310,7766637110,4/14/20,562,PPP,H AND G PLUMBING AND HEATING INC,1355 East State Road 2,LA PORTE,...,999 Other,2382 Building Equipment Contractors,2382.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1355 East State Road 2, LA PORTE, IN",NaN,NaN,NaN
367312,367312,367312,367312,7921298308,1/28/21,562,PPS,HAMILTON CROSSING ANIMAL HOSPITAL,3101 IN-32,Westfield,...,999 Other,8129 Other Personal Services,8129.0,INDIANA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"3101 IN-32, Westfield, IN",NaN,NaN,NaN
